In [10]:
!pip install sentence_transformer pypdf faiss-gpu langchain-community

!pip install langchain langchain-openai

ERROR: Could not find a version that satisfies the requirement sentence_transformer (from versions: none)
ERROR: No matching distribution found for sentence_transformer


In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
# loading the document (pdf)

loader = PyPDFLoader(file_path = '/content/drive/My Drive/qlore paper.pdf')